In [ ]:
!pip install ebooklib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ebooklib: filename=EbookLib-0.18-py3-none-any.whl size=38778 sha256=3c97b6d2cfee8417e1b82b0f55ba4eb8f6ee1e983fe7aba324f53c2a22d7fa00
  Stored in directory: /root/.cache/pip/wheels/0f/38/cc/a3728bb72a315d9d8766fb71d362136372066fc25ad838f8fa
Successfully built ebooklib


In [ ]:
import nltk
import ebooklib
from ebooklib import epub
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import os
import html
from google.colab import drive
import random

# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from html import escape

def process_text(token):
    if token.lower() not in stop_words:
        stemmed_token = stemmer.stem(token)
        #stemmed_token = token[:int(random.randint(1,len(token))*0.8)]
        extra = token[len(stemmed_token):] if len(stemmed_token) < len(token) else ''
    else:
        stemmed_token = None
        extra = ''
    res = ''
    if stemmed_token:
        res += '<b>{}</b>{}'.format(escape(stemmed_token), extra)
    else:
        res += escape(token)
    return res

In [ ]:
def transform_epub(input_epub_path):
    # Load EPUB file
    book = epub.read_epub(input_epub_path)

    # Iterate through each item in the book
    for item in book.get_items():
        # Check if the item is of type 'text'
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            # Get the content of the item
            content = item.get_content().decode('utf-8')

            # Use BeautifulSoup to parse HTML content
            soup = BeautifulSoup(content, 'html.parser')

            # Find and process each word within the HTML tags
            for tag in soup.find_all():
                if tag.name and tag.name not in ['style', 'script']:
                    # Tokenize words using NLTK
                    words = word_tokenize(tag.get_text())

                    # Process and replace each word
                    for i, word in enumerate(words):
                        transformed_word = process_text(word)
                        words[i] = transformed_word

                    # Replace the original content with the modified content
                    if tag.string is not None and ' '.join(words):
                        new_content = BeautifulSoup(' '.join(words), 'html.parser')
                        if new_content is not None:
                            tag.contents = new_content.contents



            # Update the content with preserved HTML structure
            item.content = str(soup).encode('utf-8')

    # Save the modified EPUB to the current working directory
    output_epub_path = os.path.join('your/path', 'your_book.epub')
    epub.write_epub(output_epub_path, book)

In [ ]:
drive.mount('yor/path')

# Manually specify the input EPUB file path in Google Drive
input_epub_path = 'your/path/transformed_book.epub'
book = epub.read_epub(input_epub_path)

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


In [ ]:
transform_epub(input_epub_path)